In [1]:

import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

sm = SMOTE(random_state=42)
df = pd.read_csv('Train_v2.csv')
tst = pd.read_csv('Test_v2.csv')

df['input'] ='Train'
tst['input'] = 'Test'
print(df.shape)
print(tst.shape)

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:455: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:456: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nbuser/anaconda3_420/lib/pyth

(23524, 14)
(10086, 13)


In [2]:
df = pd.concat([df,tst], axis=0)

In [3]:
import matplotlib.pyplot as plt
df.groupby(['country','bank_account'])['country'].count().plot.bar()

In [4]:
ob_fe = df.select_dtypes(include=['object'])
ob_fe =ob_fe.drop(['uniqueid'], axis=1)
ob_fe1 = pd.get_dummies(ob_fe)
int_fe = df.select_dtypes(include=['float64','int64'])
fin = pd.concat([df['uniqueid'],int_fe,ob_fe1], axis=1)

#fin = pd.concat([fin[fin['input_Train']==1],fin[fin['input_Test']==1]],axis=0)

train = fin[fin['input_Train']==1]
test = fin[fin['input_Test']==1]

In [38]:
X=train
X = X.drop(['uniqueid','bank_account_No','bank_account_Yes','location_type_Rural','cellphone_access_No','gender_of_respondent_Female','input_Test', 'input_Train'],axis=1)
Y= train['bank_account_Yes']

# X_res, Y_res = sm.fit_resample(X, Y)

# X=pd.DataFrame(X_res, columns=X.columns)
# Y=pd.Series(Y_res)

sclar = StandardScaler()
sclar.fit(X)
X=sclar.transform(X)


In [41]:
X_test=test
X_test = X_test.drop(['uniqueid','bank_account_No','bank_account_Yes','location_type_Rural','cellphone_access_No','gender_of_respondent_Female','input_Test', 'input_Train'],axis=1)
Y_test= test['bank_account_Yes']


sclar = StandardScaler()
sclar.fit(X_test)
X_test=sclar.transform(X_test)
y_pred_test = pd.Series(clf.predict(X_test))

In [75]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
train_X, val_X, train_y, val_y = train_test_split(X, Y, random_state=1,test_size=0.3)

clf = RandomForestClassifier(n_estimators=500, max_depth=8, random_state=21 ,oob_score=True)
clf.fit(train_X,train_y)
y_pred = clf.predict(train_X)

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_val_score

print(classification_report(train_y, y_pred))
print(accuracy_score(train_y, y_pred))
print(cross_val_score(clf, train_X, train_y, cv=5)) 
print(accuracy_score(val_y,clf.predict(val_X)))

y_pred_test_rf = clf.predict(X_test)

#feat_importances = pd.Series(clf.feature_importances_, index=X.columns)
#feat_importances.nlargest(25)

              precision    recall  f1-score   support

           0       0.90      0.99      0.94     14146
           1       0.85      0.30      0.44      2320

    accuracy                           0.89     16466
   macro avg       0.87      0.64      0.69     16466
weighted avg       0.89      0.89      0.87     16466

0.893598931130815
[0.88160291 0.88338901 0.89462496 0.8870331  0.88733677]
0.8826863134032303


In [76]:
import xgboost as xgb
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2

# fit model no training data
model = xgb.XGBClassifier(num_round=50, max_depth=8, objective='binary:logistic',eta=1)
model.fit(train_X, train_y)
y_pred = model.predict(train_X)

print(classification_report(train_y, y_pred))
print(accuracy_score(train_y, y_pred))
print(cross_val_score(model, train_X, train_y, cv=5)) 


y_pred_test_xgb = model.predict(X_test)
print(sum(y_pred_test))
#predictions = [round(value) for value in y_pred]

              precision    recall  f1-score   support

           0       0.92      0.98      0.95     14146
           1       0.83      0.49      0.61      2320

    accuracy                           0.91     16466
   macro avg       0.88      0.73      0.78     16466
weighted avg       0.91      0.91      0.90     16466

0.9136402283493259
[0.88251366 0.88490738 0.89371394 0.88369268 0.88217431]
616


In [78]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(train_X, train_y)
y_pred = clf.predict(train_X)


print(classification_report(train_y, y_pred))
print(accuracy_score(train_y, y_pred))
print(cross_val_score(clf, train_X, train_y, cv=5)) 


y_pred_test_svm = clf.predict(X_test)
print(sum(y_pred_test))


              precision    recall  f1-score   support

           0       0.90      0.98      0.94     14146
           1       0.79      0.35      0.48      2320

    accuracy                           0.89     16466
   macro avg       0.85      0.67      0.71     16466
weighted avg       0.89      0.89      0.88     16466

0.8948742864083566
[0.87978142 0.88521105 0.8988764  0.88855147 0.88733677]
616


In [80]:
y_pred_test_xgb

array([1, 1, 0, ..., 0, 0, 0], dtype=uint8)

In [82]:
score =pd.DataFrame(pd.concat([pd.Series(y_pred_test_rf),pd.Series(y_pred_test_xgb),pd.Series(y_pred_test_svm)],axis=1))

In [87]:
submission= pd.Series(tst['uniqueid'] + " x "+ tst['country'])
submission = pd.concat([submission,score],axis=1)
submission.rename(columns={0:'uniqueid',1:'bank_account'},inplace=True)

In [88]:
submission.to_csv('SubmissionFile.csv', index=False)

In [82]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [86]:
model = Sequential()
model.add(Dense(128, activation="relu", input_shape = (train_X.shape[1],))) # Hidden Layer 1 that receives the Input from the Input Layer

model.add(Dense(64, activation="relu")) # Hidden Layer 2
model.add(Dropout(0.2))

model.add(Dense(32, activation="relu")) # Hidden Layer 3
model.add(Dropout(0.2))

model.add(Dense(16, activation="relu")) # Hidden Layer 4
model.add(Dropout(0.2))


model.add(Dense(1, activation="sigmoid")) # Outout Layer

model.summary()

model.compile(optimizer='adam', loss = "binary_crossentropy", metrics = ['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 128)               4864      
_________________________________________________________________
dense_12 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_8 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 16)                528       
_________________________________________________________________
dropout_9 (Dropout)          (None, 16)                0         
__________